In [ ]:
"""
SYS 660 - College Decision Support System
Final Project Code
Authors: Stephanie McDonough, Anthony Rizzuto, Justin Baumann
Date: 04/15/2025
"""

In [ ]:
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output
from sklearn.preprocessing import MinMaxScaler

# Load and prepare data
df = pd.read_csv('College Data Full-US.csv', low_memory=False)

# Clean and parse majors
def parse_programs(programs):
    if pd.isna(programs) or programs.strip() == "N/A":
        return set()
    return set(map(str.strip, programs.split(',')))

df["Parsed Majors"] = df["Bachelor's Programs Offered"].apply(parse_programs)

# Collect all majors across institutions
def get_all_majors(df):
    majors = set()
    for major_set in df["Parsed Majors"]:
        majors.update(major_set)
    return sorted(majors)

# Widgets
state_options = ['All States'] + sorted(df["State abbreviation (HD2023)"].dropna().unique().tolist())
state_widget = widgets.SelectMultiple(
    options= ['All States'] + sorted(df["State abbreviation (HD2023)"].dropna().unique().tolist()),
    value=[],
    description='Select State(s):',
    layout=widgets.Layout(width='400px', height='420px')
)

city_widget = widgets.Text(
    value='',
    description='City (optional):',
    layout=widgets.Layout(width='400px')
)

major_options = ['All Majors'] + get_all_majors(df)
major_widget = widgets.SelectMultiple(
    options= ['All Majors'] + get_all_majors(df),
    value=[],
    description='Select Major(s):',
    layout=widgets.Layout(width='400px', height='220px')
)

max_tuition_widget = widgets.IntSlider(
    value=60000, min=0, max=100000, step=1000,
    description='Max Tuition ($):',
    layout=widgets.Layout(width='400px')
)

min_acceptance_widget = widgets.FloatSlider(
    value=0, min=0, max=100, step=0.1,
    description='Min Acceptance Rate (%):',
    layout=widgets.Layout(width='400px')
)

min_earnings_widget = widgets.IntSlider(
    value=10000, min=0, max=200000, step=1000,
    description='Min Median Earnings ($):',
    layout=widgets.Layout(width='400px')
)

size_widget = widgets.IntRangeSlider(
    value=(0,12000),
    min=0,
    max=200000,
    step=1000,
    description='Enrollment Range:',
    layout=widgets.Layout(width='400px')
)


# Athletic program participation widgets
football_widget = widgets.Dropdown(
    options=['No Preference', 'Yes', 'No'],
    value='No Preference',
    description='Football:',
    layout=widgets.Layout(width='300px')
)

basketball_widget = widgets.Dropdown(
    options=['No Preference', 'Yes', 'No'],
    value='No Preference',
    description='Basketball:',
    layout=widgets.Layout(width='300px')
)

baseball_widget = widgets.Dropdown(
    options=['No Preference', 'Yes', 'No'],
    value='No Preference',
    description='Baseball:',
    layout=widgets.Layout(width='300px')
)

track_widget = widgets.Dropdown(
    options=['No Preference', 'Yes', 'No'],
    value='No Preference',
    description='Track/XC:',
    layout=widgets.Layout(width='300px')
)


def on_state_change(change):
    if 'All States' in change['new']:
        with state_widget.hold_trait_notifications():
            state_widget.value = tuple([s for s in state_options if s != 'All States'])

def on_major_change(change):
    if 'All Majors' in change['new']:
        with major_widget.hold_trait_notifications():
            major_widget.value = tuple([m for m in major_options if m != 'All Majors'])

state_widget.observe(on_state_change, names='value')
major_widget.observe(on_major_change, names='value')

# Weights
tuition_weight = widgets.FloatSlider(value=5, min=0, max=10, step=0.5, description='Tuition Importance:', layout=widgets.Layout(width='300px'))
acceptance_weight = widgets.FloatSlider(value=5, min=0, max=10, step=0.5, description='Acceptance Rate Importance:', layout=widgets.Layout(width='300px'))
earnings_weight = widgets.FloatSlider(value=5, min=0, max=10, step=0.5, description='Earnings Importance:', layout=widgets.Layout(width='300px'))
major_weight = widgets.FloatSlider(value=5, min=0, max=10, step=0.5, description='Major Match Importance:', layout=widgets.Layout(width='300px'))
size_weight = widgets.FloatSlider(value=5, min=0, max=10, step=0.5,description='School Size Importance:', layout=widgets.Layout(width='300px'))

# Score calculation
def calculate_recommendation_scores(filtered_df, weights, selected_majors):
    scaler = MinMaxScaler()
    total_weight = sum(weights.values())
    normalized_weights = {k: v / total_weight if total_weight else 1/4 for k, v in weights.items()}

    matrix = filtered_df[[
        'Published in-state tuition and fees 2023-24 (IC2023_AY)',
        'Percent admitted - total (DRVADM2023)',
        'MD_EARN_WNE_P10',
        'Undergraduate enrollment (DRVEF2023)'
    ]].copy()

    def has_major(major_set):
        if not selected_majors:
            return 1
        return int(any(m in major_set for m in selected_majors))

    matrix['major_score'] = filtered_df["Parsed Majors"].apply(has_major)

    for col, norm_col, invert in [
        ('Published in-state tuition and fees 2023-24 (IC2023_AY)', 'tuition_norm', True),
        ('Percent admitted - total (DRVADM2023)', 'acceptance_norm', False),
        ('MD_EARN_WNE_P10', 'earnings_norm', False),
        ('Undergraduate enrollment (DRVEF2023)', 'size_norm', False)
    ]:
        if col in matrix.columns and matrix[col].notna().sum() > 1:
            norm = scaler.fit_transform(matrix[[col]].fillna(matrix[col].mean()))
            matrix[norm_col] = (1 - norm if invert else norm).flatten()
        else:
            matrix[norm_col] = 1.0

    matrix['score'] = (
        matrix['tuition_norm'] * normalized_weights['tuition'] +
        matrix['acceptance_norm'] * normalized_weights['acceptance'] +
        matrix['earnings_norm'] * normalized_weights['earnings'] +
        matrix['major_score'] * normalized_weights['major'] +
        matrix['size_norm'] * normalized_weights['size']
    )
    return matrix['score']

# Update display
def update_recommendations(change):
    with out:
        clear_output(wait=True)
        filtered = df.copy()

        if state_widget.value:
            selected_states = [s for s in state_widget.value if s != 'All States']
            if selected_states:
              filtered = filtered[filtered["State abbreviation (HD2023)"].isin(selected_states)]

        if city_widget.value:
            filtered = filtered[filtered["City location of institution (HD2023)"].str.contains(city_widget.value, case=False, na=False)]

        if 'Published in-state tuition and fees 2023-24 (IC2023_AY)' in filtered.columns:
            filtered = filtered[
              filtered['Published in-state tuition and fees 2023-24 (IC2023_AY)'].fillna(float('inf')) <= max_tuition_widget.value]

        if 'Percent admitted - total (DRVADM2023)' in filtered.columns:
            filtered = filtered[
              filtered['Percent admitted - total (DRVADM2023)'].fillna(0) >= min_acceptance_widget.value]

        min_size, max_size = size_widget.value
        if 'Undergraduate enrollment (DRVEF2023)' in filtered.columns:
            filtered = filtered[
                filtered['Undergraduate enrollment (DRVEF2023)'].fillna(0).between(min_size, max_size)
            ]


        if 'MD_EARN_WNE_P10' in filtered.columns:
            filtered = filtered[
              filtered['MD_EARN_WNE_P10'].fillna(0) >= min_earnings_widget.value]

        selected_majors = [m for m in major_widget.value if m != 'All Majors']
        if selected_majors:
            filtered = filtered[filtered["Parsed Majors"].apply(lambda x: any(m in x for m in selected_majors))]


        if filtered.empty:
            display(pd.DataFrame({"Message": ["No colleges match your criteria."]}))
            return

        # Athletic program filters
        sport_filters = {
          'NCAA/NAIA member for football (IC2023)': football_widget.value,
          'NCAA/NAIA member for basketball (IC2023)': basketball_widget.value,
          'NCAA/NAIA member for baseball (IC2023)': baseball_widget.value,
          'NCAA/NAIA member for cross country/track (IC2023)': track_widget.value}

        for col, val in sport_filters.items():
          if val != 'No Preference':
            filtered = filtered[filtered[col].fillna('N/A').str.lower() == val.lower()]

        print(f"{len(filtered)} colleges matched your filters.")

        weights = {
            'tuition': tuition_weight.value,
            'acceptance': acceptance_weight.value,
            'earnings': earnings_weight.value,
            'major': major_weight.value,
            'size': size_weight.value
        }

        filtered = filtered.copy()
        filtered['recommendation_score'] = calculate_recommendation_scores(filtered, weights, selected_majors)
        filtered = filtered.sort_values('recommendation_score', ascending=False)

        # Add sport badges (emoji tags)
        def get_sport_badges(row):
          badges = []
          if str(row.get('NCAA/NAIA member for football (IC2023)', '')).lower() == 'yes':
              badges.append('🏈')
          if str(row.get('NCAA/NAIA member for basketball (IC2023)', '')).lower() == 'yes':
              badges.append('🏀')
          if str(row.get('NCAA/NAIA member for baseball (IC2023)', '')).lower() == 'yes':
              badges.append('⚾')
          if str(row.get('NCAA/NAIA member for cross country/track (IC2023)', '')).lower() == 'yes':
              badges.append('🏃')
          return ' '.join(badges)


        filtered['Athletics'] = filtered.apply(get_sport_badges, axis=1)

        display_cols = [
            'Institution Name',
            'City location of institution (HD2023)',
            'State abbreviation (HD2023)',
            'recommendation_score',
            'Athletics',
            'Published in-state tuition and fees 2023-24 (IC2023_AY)',
            'Percent admitted - total (DRVADM2023)',
            'Undergraduate enrollment (DRVEF2023)',
            'MD_EARN_WNE_P10',
            'MN_EARN_WNE_P10'
        ]
        filtered['recommendation_score'] = filtered['recommendation_score'].round(4)
        display(filtered[display_cols].head(20).style.format({
            'recommendation_score': '{:.4f}',
            'Published in-state tuition and fees 2023-24 (IC2023_AY)': '${:,.0f}',
            'Percent admitted - total (DRVADM2023)': '{:.1f}%',
            'Undergraduate enrollment (DRVEF2023)' : '{:.0f}',
            'MD_EARN_WNE_P10': '${:,.0f}',
            'MN_EARN_WNE_P10': '${:,.0f}'
        }))

# UI setup
filters = widgets.VBox([
    widgets.HTML('<h3>Filter Criteria</h3>'),
    state_widget,
    city_widget,
    major_widget,
    max_tuition_widget,
    min_acceptance_widget,
    size_widget,
    min_earnings_widget,
    widgets.HTML('<h4>Athletic Participation (NCAA/NAIA)</h4>'),
    football_widget,
    basketball_widget,
    baseball_widget,
    track_widget
])

weights = widgets.VBox([
    widgets.HTML('<h3>Criteria Importance (0-10)</h3>'),
    tuition_weight,
    acceptance_weight,
    earnings_weight,
    major_weight,
    size_weight
])

ui = widgets.Tab(children=[filters, weights])
ui.set_title(0, 'Filters')
ui.set_title(1, 'Scoring Weights')

out = widgets.Output()

for w in [
    state_widget, city_widget, major_widget,
    max_tuition_widget, min_acceptance_widget, min_earnings_widget,
    tuition_weight, acceptance_weight, earnings_weight, major_weight,football_widget, basketball_widget, baseball_widget, track_widget, size_widget
]:
    w.observe(update_recommendations, names='value')

display(ui, out)
update_recommendations(None)

Output()